# Watsonを使った関係抽出

このノートブックでは、IBM Watsonを用いて関係抽出をします。[無料枠](https://www.ibm.com/cloud/free)があるので、試すのには十分でしょう。

1. アカウントを作成
2. Natural Language Understandingを作成
3. 「管理」ページで、 「表示」をクリックして資格情報を表示
4. 「API キー」の値と「URL」の値をコピー


## パッケージのインストール

In [1]:
!pip install -q ibm-watson==5.3.0

     |████████████████████████████████| 412 kB 8.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 45 kB 2.8 MB/s 
     |████████████████████████████████| 68 kB 7.0 MB/s 
  Created wheel for ibm-watson: filename=ibm_watson-5.3.0-py3-none-any.whl size=408872 sha256=2c145dd642bfa9801b534e9b8cf64253e21d067a2e4409fc8c4f0a9023b873cb
  Stored in directory: /root/.cache/pip/wheels/21/d9/82/4ce5b94730bc4f1f7b4c6384f72964b9b8f79fcc125bb8085c
  Created wheel for ibm-cloud-sdk-core: filename=ibm_cloud_sdk_core-3.11.3-py3-none-any.whl size=74751 sha256=1148cc6d0d1bfd31119e2ab2f029eb90e3502f1a4a84505fd798f04ca95d88df
  Stored in directory: /root/.cache/pip/wheels/93/85/b2/7c2428617824a175293f7957d4a20b4975eb0ddbef52800006
Successfully built ibm-watson ibm-cloud-sdk-core


## 関係抽出

コピーしたAPIキーとURLを以下のコードに貼り付けて実行してみましょう。

In [5]:
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_watson.natural_language_understanding_v1 import Features, RelationsOptions
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

# Authentication via IAM
API_KEY = "{API KEY}"
URL = "{URL}"
authenticator = IAMAuthenticator(API_KEY)
service = NaturalLanguageUnderstandingV1(
    version="2018-03-16",
    authenticator=authenticator
)
service.set_service_url(URL)
response = service.analyze(
    text="Leonardo DiCaprio won Best Actor in a Leading Role for his performance.",
    features=Features(relations=RelationsOptions()),
).get_result()
print(json.dumps(response, indent=2))

{
  "usage": {
    "text_units": 1,
    "text_characters": 71,
    "features": 1
  },
  "relations": [
    {
      "type": "awardedTo",
      "sentence": "Leonardo DiCaprio won Best Actor in a Leading Role for his performance.",
      "score": 0.680715,
      "arguments": [
        {
          "text": "Best Actor",
          "location": [
            22,
            32
          ],
          "entities": [
            {
              "type": "EntertainmentAward",
              "text": "Best Actor"
            }
          ]
        },
        {
          "text": "Leonardo DiCaprio",
          "location": [
            0,
            17
          ],
          "entities": [
            {
              "type": "Person",
              "text": "Leonardo DiCaprio"
            }
          ]
        }
      ]
    }
  ],
  "language": "en"
}


In [6]:
mytext = "Satya Narayana Nadella currently serves as the Chief Executive Officer (CEO) of Microsoft."

In [7]:
response = service.analyze(
    text=mytext, features=Features(relations=RelationsOptions())
).get_result()
print(json.dumps(response, indent=2))

{
  "usage": {
    "text_units": 1,
    "text_characters": 90,
    "features": 1
  },
  "relations": [
    {
      "type": "employedBy",
      "sentence": "Satya Narayana Nadella currently serves as the Chief Executive Officer (CEO) of Microsoft.",
      "score": 0.48706,
      "arguments": [
        {
          "text": "CEO",
          "location": [
            72,
            75
          ],
          "entities": [
            {
              "type": "Person",
              "text": "Satya Narayana Nadella"
            }
          ]
        },
        {
          "text": "Microsoft",
          "location": [
            80,
            89
          ],
          "entities": [
            {
              "type": "Organization",
              "text": "Microsoft",
              "disambiguation": {
                "subtype": [
                  "Commercial"
                ]
              }
            }
          ]
        }
      ]
    }
  ],
  "language": "en"
}


In [8]:
for item in response["relations"]:
    print(item["type"])
    for subitem in item["arguments"]:
        print(subitem["entities"])

employedBy
[{'type': 'Person', 'text': 'Satya Narayana Nadella'}]
[{'type': 'Organization', 'text': 'Microsoft', 'disambiguation': {'subtype': ['Commercial']}}]


In [9]:
mytext2 = "Nadella was born in Hyderabad. His father, Bukkapuram Nadella Yugandher, was a civil servant who worked for the Indian Administrative Service of the Government of India. His mother was a Sanskrit scholar. "
response = service.analyze(
    text=mytext2, features=Features(relations=RelationsOptions())
).get_result()
for item in response["relations"]:
    print(item["type"])
    for subitem in item["arguments"]:
        print(subitem["entities"])

bornAt
[{'type': 'Person', 'text': 'Nadella'}]
[{'type': 'GeopoliticalEntity', 'text': 'Hyderabad'}]
parentOf
[{'type': 'Person', 'text': 'Bukkapuram Nadella Yugandher'}]
[{'type': 'Person', 'text': 'Nadella'}]
employedBy
[{'type': 'Person', 'text': 'Administrative'}]
[{'type': 'GeopoliticalEntity', 'text': 'Indian', 'disambiguation': {'subtype': ['Country']}}]
managerOf
[{'type': 'Person', 'text': 'Administrative'}]
[{'type': 'Organization', 'text': 'Service', 'disambiguation': {'subtype': ['Government']}}]
parentOf
[{'type': 'Person', 'text': 'scholar'}]
[{'type': 'Person', 'text': 'Nadella'}]
locatedAt
[{'type': 'Person', 'text': 'scholar'}]
[{'type': 'Facility', 'text': 'Sanskrit'}]
locatedAt
[{'type': 'Person', 'text': 'scholar'}]
[{'type': 'Facility', 'text': 'Sanskrit'}]


In [10]:
mytext3 = """Nadella attended the Hyderabad Public School, Begumpet [12] before receiving
a bachelor's in electrical engineering[13] from the Manipal Institute of Technology
(then part of Mangalore University) in Karnataka in 1988."""
response = service.analyze(
    text=mytext3, features=Features(relations=RelationsOptions())
).get_result()
for item in response["relations"]:
    print(item["type"])
    for subitem in item["arguments"]:
        print(subitem["entities"])

employedBy
[{'type': 'Person', 'text': 'Nadella'}]
[{'type': 'Organization', 'text': 'Hyderabad Public School', 'disambiguation': {'subtype': ['Commercial']}}]
awardedTo
[{'type': 'Degree', 'text': 'bachelor'}]
[{'type': 'Person', 'text': 'Nadella'}]
educatedAt
[{'type': 'Person', 'text': 'Nadella'}]
[{'type': 'Organization', 'text': 'Manipal Institute of Technology', 'disambiguation': {'subtype': ['Educational']}}]
educatedAt
[{'type': 'Person', 'text': 'Nadella'}]
[{'type': 'Organization', 'text': 'Mangalore University', 'disambiguation': {'subtype': ['Educational']}}]
awardedBy
[{'type': 'Degree', 'text': 'bachelor'}]
[{'type': 'Organization', 'text': 'Manipal Institute of Technology', 'disambiguation': {'subtype': ['Educational']}}]
basedIn
[{'type': 'Organization', 'text': 'Mangalore University', 'disambiguation': {'subtype': ['Educational']}}]
[{'type': 'GeopoliticalEntity', 'text': 'Karnataka'}]
